In [10]:
from datetime import datetime
from pathlib import Path
import pandas as pd # type: ignore

In [11]:
def normalize_time(time_str):
    try:
        # Split time into components
        h, m, s = map(int, time_str.split(":"))
        # Normalize hours to wrap around at 24
        h %= 24
        # Return normalized time as a string
        return f"{h:02}:{m:02}:{s:02}"
    except Exception as e:
        print(f"Invalid time format: {time_str}")
        return None

In [12]:
# Read txt files
input = '../data_in'
output = '../data_out'
data_set = 'mdb-2253'

directory = Path(output)

for file_path in directory.iterdir():
    if file_path.is_file():  # Check if it's a file
        file_path.unlink()  # Delete the file
        # print(f"Deleted: {file_path}")

routes = pd.read_csv(input + '/' + data_set + '/routes.txt', sep=',')
# print(routes)
trips = pd.read_csv(input + '/' + data_set + '/trips.txt', sep=',')
# print(trips)
stop_times = pd.read_csv(input + '/' + data_set + '/stop_times.txt', sep=',')
# print(stop_times)
stops = pd.read_csv(input + '/' + data_set + '/stops.txt', sep=',')
# print(stops)

In [13]:
# Filter for bus 510 Spadina
route_short_name = 510
route_id = routes.loc[routes['route_short_name'] == route_short_name, 'route_id']
print(route_id)

132    510
Name: route_id, dtype: int64


In [14]:
# Get all trip_ids with route_id = 510
route_trips = trips.loc[trips['route_id'] == route_id.iloc[0], 'trip_id']
print(route_trips)

105056    144643080
105057     14359010
105058    144180040
105059    143674050
105060    144363080
            ...    
109468      8560010
109469     85116040
109470     85077040
109471     84907050
109472     84838020
Name: trip_id, Length: 4417, dtype: int64


In [15]:
# Get route stop times from stop_times csv
route_stop_times = stop_times[stop_times['trip_id'].isin(route_trips)]
print(route_stop_times)

           trip_id arrival_time departure_time  stop_id  stop_sequence  \
515          19040     08:51:00       08:51:00     4502              1   
516          19040     08:53:15       08:53:15      315              2   
517          19040     08:53:58       08:53:58     4264              3   
518          19040     08:55:14       08:55:14     4265              4   
519          19040     08:55:49       08:55:49    18794              5   
...            ...          ...            ...      ...            ...   
6267274  196539050     06:58:45       06:58:45    42405              9   
6267275  196539050     06:59:17       06:59:17    42388             10   
6267276  196539050     07:00:12       07:00:12    42402             11   
6267277  196539050     07:00:43       07:00:43    42392             12   
6267278  196539050     07:01:00       07:01:00    30764             13   

                   stop_headsign  pickup_type  drop_off_type  \
515      510 SPADINA SPADINA STN            0  

In [16]:
# Join with stops csv to get stop information
route_stop_times = route_stop_times.merge(stops, on='stop_id', how='left')
# print(route_stop_times)

In [17]:
# Apply time normalization (for >24 hours) and convert to datetime format
route_stop_times["arrival_time"] = route_stop_times["arrival_time"].apply(normalize_time)
route_stop_times["arrival_time"] = pd.to_datetime(route_stop_times["arrival_time"], format="%H:%M:%S").dt.time

# Take all trip_ids with arrival_time +/- 1 minutes to current time
current_time = pd.Timestamp.now()

# Combine arrival_time with today's date to create datetime objects
today = pd.Timestamp.now().normalize()  # Get today's date at 00:00:00
route_stop_times["arrival_datetime"] = route_stop_times["arrival_time"].apply(
    lambda t: datetime.combine(today, t)
)

# Filter rows to 1 minute in future
time_diff = (route_stop_times["arrival_datetime"] - current_time)
route_stop_times = route_stop_times[(time_diff <= pd.Timedelta(minutes=1)) & (time_diff > pd.Timedelta(0))]

In [18]:
# Sort by arrival_time and stop_sequence
route_stop_times_sorted = (
    route_stop_times.groupby('trip_id', group_keys=False)
    .apply(lambda group: group.sort_values(by=['trip_id', 'arrival_time', 'stop_sequence'], ascending=[False, True, True]))
)

print(route_stop_times_sorted)

# Display arrival/departure times and lat/lon coordinates for each stop 
result_line = route_stop_times_sorted[['trip_id', 'stop_id', 'stop_name', 'arrival_time', 'departure_time', 'stop_lat', 'stop_lon', 'stop_sequence']]

# Count unique trip_ids just out of curiosity
unique_trip_count = route_stop_times["trip_id"].nunique()
print(unique_trip_count)

# Save to csv
result_line.reset_index(drop = True, inplace = True)
result_line.to_csv(output + '/' + str(route_short_name) + "-static.csv")

         trip_id arrival_time departure_time  stop_id  stop_sequence  \
5549    13572010     18:39:00       18:39:00     4502              1   
6440    16325040     18:39:14       18:39:14    42386             16   
6871    17227010     18:39:09       18:39:09    42387             18   
7533    18653070     18:38:54       18:38:54    42400             10   
7883    19754040     18:39:24       18:39:24    42385              2   
...          ...          ...            ...      ...            ...   
75469  188924080     18:38:46       18:38:46     4260             18   
76713  192276050     18:39:09       18:39:09    42387             18   
76907  192504020     18:38:46       18:38:46     4260             18   
76954  192687080     18:39:37       18:39:37    42405             19   
77783  195062070     18:38:51       18:38:51    42390              9   

                    stop_headsign  pickup_type  drop_off_type  \
5549      510 SPADINA SPADINA STN            0              0   
6440 

/tmp/ipykernel_204090/1838574608.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.sort_values(by=['trip_id', 'arrival_time', 'stop_sequence'], ascending=[False, True, True]))
